In [ ]:
!apt update
!apt install cmake unzip zip ffmpeg espeak python3.8 python3.8-venv python3.8-dev portaudio19-dev -y
!update-alternatives --install /usr/bin/python python /usr/bin/python3.8 1
!update-alternatives --config python
!python3 --version
!wget https://bootstrap.pypa.io/get-pip.py
!python3.8 get-pip.py

In [ ]:
!git clone https://github.com/FENRlR/MB-iSTFT-VITS2.git
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 --index-url https://download.pytorch.org/whl/cu117

In [ ]:
%cd MB-iSTFT-VITS2
with open('requirements.txt', 'r') as file:
    lines = file.readlines()

lines = [line for line in lines if "pyopenjtalk==0.2.0" not in line]

with open('requirements.txt', 'w') as file:
    file.writelines(lines)

In [ ]:
!pip install -r requirements.txt
!pip install -U pyopenjtalk==0.2.0 --no-build-isolation

In [ ]:
%cd monotonic_align
!rm -rf ./monotonic_align
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..

In [ ]:
python preprocess.py --text_index 1 --filelists filelists/filelist_train.txt filelists/filelist_val.txt --text_cleaners korean_cleaners

In [ ]:
%cd /workspace/MB-iSTFT-VITS2/
import json

file_path = 'configs/mb_istft_vits2_base.json'

batch_size = 32
training_files_path = "filelists/filelist_train.txt.cleaned"
validation_files_path = "filelists/filelist_val.txt.cleaned"
sampling_rate = 22050

with open(file_path, 'r') as f:
    config = json.load(f)

config["train"]["batch_size"] = batch_size
config["data"]["training_files"] = training_files_path
config["data"]["validation_files"] = validation_files_path
config["data"]["sampling_rate"] = sampling_rate

with open(file_path, 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
%cd /workspace/
!mkdir model

In [ ]:
%cd /workspace/MB-iSTFT-VITS2/
!python train.py -c /workspace/MB-iSTFT-VITS2/configs/mb_istft_vits2_base.json -m /workspace/model